In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# Deep Neural Networks 
## Session 11:Lecture

## Working with Tensorflow 
- One hidden layer
- Fashion MNIST
- Additional Code (Plot loss curve beautified)

</p><img src='../../prasami_images/prasami_color_tutorials_small.png' width='400' alt="By Pramod Sharma : pramod.sharma@prasami.com" align="left"/>

In [ ]:
# Lets import some libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler

import tensorflow as tf

%matplotlib inline

<div class="alert alert-block alert-info" style="font-family: Arial; font-size:1.1em;">
    <b>Notes:</b>
    - `EPOCHS` to 20.
    - Add `altName = 'no_frils'`
</div>

In [ ]:
# Some basic parameters

inpDir = '../../input' # location where input data is stored
outDir = '../output' # location to store outputs
modelDir = '../models'
subDir = 'fashion_mnist'

altName = 'no_frils'

RANDOM_STATE = 24 # for initialization ----- REMEMBER: to remove at the time of promotion to production

np.random.seed(RANDOM_STATE) # Set Random Seed for reproducible  results
tf.random.set_seed(RANDOM_STATE) # setting for Tensorflow as well


EPOCHS = 10 # number of epochs
ALPHA = 0.01 # learning rate

# Set parameters for decoration of plots
params = {'legend.fontsize' : 'large',
          'figure.figsize'  : (15,15),
          'axes.labelsize'  : 'x-large',
          'axes.titlesize'  :'x-large',
          'xtick.labelsize' :'large',
          'ytick.labelsize' :'large',
         }

CMAP = plt.cm.brg

plt.rcParams.update(params) # update rcParams

## Helper Functions
### Plot loss curve
<div class="alert alert-block alert-warning" style="font-family: Arial; font-size:1.1em;">
    <b>Notes:</b>Following code of plotting loss curve has additional beautification.
</div>

In [ ]:
###-----------------------------------
### Function to plot Loss Curve
###-----------------------------------

def fn_plot_tf_hist(hist_df : pd.DataFrame):
    '''
    Args:
      hist_df : pandas Dataframe with four columns
                For 'x' values, we will use index
    '''
    fig, axes = plt.subplots(1,2 , figsize = (15,6))

    # properties  matplotlib.patch.Patch 
    props = dict(boxstyle='round', facecolor='aqua', alpha=0.4)
    facecolor = 'cyan'
    fontsize=12
    
    # Get columns by index to eliminate any column naming error
    y1 = hist_df.columns[0]
    y2 = hist_df.columns[1]
    y3 = hist_df.columns[2]
    y4 = hist_df.columns[3]

    # Where was min loss
    best = hist_df[hist_df[y3] == hist_df[y3].min()]
    
    ax = axes[0]

    hist_df.plot(y = [y1,y3], ax = ax, colormap=CMAP)


    # little beautification
    txtFmt = "Loss: \n  train: {:6.4f}\n   test: {:6.4f}"
    txtstr = txtFmt.format(hist_df.iloc[-1][y1],
                           hist_df.iloc[-1][y3]) #text to plot
    
    # place a text box in upper middle in axes coords
    ax.text(0.3, 0.95, txtstr, transform=ax.transAxes, fontsize=fontsize,
            verticalalignment='top', bbox=props)

    # Mark arrow at lowest
    ax.annotate(f'Min: {best[y3].to_numpy()[0]:6.4f}', # text to print
                xy=(best.index.to_numpy(), best[y3].to_numpy()[0]), # Arrow start
                xytext=(best.index.to_numpy()-1, best[y3].to_numpy()[0]), # location of text 
                fontsize=fontsize, va='bottom', ha='right',bbox=props, # beautification of text
                arrowprops=dict(facecolor=facecolor, shrink=0.05)) # arrow

    # Draw vertical line at best value
    ax.axvline(x = best.index.to_numpy(), color = 'green', linestyle='-.', lw = 3);

    ax.set_xlabel("Epochs")
    ax.set_ylabel(y1.capitalize())
    ax.set_title('Errors')
    ax.grid();
    ax.legend(loc = 'upper left') # model legend to upper left

    ax = axes[1]

    hist_df.plot( y = [y2, y4], ax = ax, colormap=CMAP)
    
    # little beautification
    txtFmt = "Accuracy: \n  train: {:6.4f}\n  test:  {:6.4f}"
    txtstr = txtFmt.format(hist_df.iloc[-1][y2],
                           hist_df.iloc[-1][y4]) #text to plot

    # place a text box in upper middle in axes coords
    ax.text(0.3, 0.2, txtstr, transform=ax.transAxes, fontsize=fontsize,
            verticalalignment='top', bbox=props)

    # Mark arrow at lowest
    ax.annotate(f'Best: {best[y4].to_numpy()[0]:6.4f}', # text to print
                xy=(best.index.to_numpy(), best[y4].to_numpy()[0]), # Arrow start
                xytext=(best.index.to_numpy()-1, best[y4].to_numpy()[0]), # location of text 
                fontsize=fontsize, va='bottom', ha='right',bbox=props, # beautification of text
                arrowprops=dict(facecolor=facecolor, shrink=0.05)) # arrow
    
    
    # Draw vertical line at best value
    ax.axvline(x = best.index.to_numpy(), color = 'green', linestyle='-.', lw = 3);

    ax.set_xlabel("Epochs")
    ax.set_ylabel(y2.capitalize())
    ax.grid()
    ax.legend(loc = 'lower left')
    
    plt.tight_layout()

### Settings so that Tensorflow can not Hog all the GPU memory

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU') 

if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

## Load Fashion MNIST

In [ ]:
train_df = pd.read_csv(os.path.join(inpDir, 
                                    subDir, 
                                    'fashion-mnist_train.csv'), header=0)

test_df = pd.read_csv(os.path.join(inpDir, 
                                    subDir, 
                                    'fashion-mnist_test.csv'), header=0)

In [ ]:
test_df.head()

In [ ]:
train_df.describe()

In [ ]:
#  Split the data in training and test sets to measure performance of the model.
X_train = train_df.drop('label', axis = 1).to_numpy()
X_test = test_df.drop('label', axis = 1).to_numpy()
y_train = train_df['label'].to_numpy()
y_test = test_df['label'].to_numpy()

In [ ]:
X_train = X_train/255.

X_test = X_test/255.

In [ ]:
X_train

<div class="alert alert-block alert-warning" style="font-family: Arial; font-size:1.1em;">
    <b>Notes:</b>Added class names.
</div>

In [ ]:
class_names = {0: 'T-shirt/top',
               1: 'Trouser',
               2: 'Pullover',
               3: 'Dress',
               4: 'Coat',
               5: 'Sandal', 
               6: 'Shirt',
               7: 'Sneaker', 
               8: 'Bag', 
               9: 'Ankle boot'}

## Sample data plot

## Prepare model

Build the `tf.keras.Sequential` model by stacking layers from `tf.keras.layers.Dense`. Activations can be given as string or from `tf.keras.activations.tanh` 

<div class="alert alert-block alert-info" style="font-family: Arial; font-size:1.1em;">
    <b>Notes:</b>Change it to `Relu`.
</div>

In [ ]:
# Define Model Calling class Sequential and pass all layers as a list

model = tf.keras.Sequential([
    
    tf.keras.layers.Dense(196, activation='tanh'), # Alter : tf.keras.activations.tanh
    
    tf.keras.layers.Dense(49, activation='tanh'), # Alter : tf.keras.activations.tanh
    
    tf.keras.layers.Dense(10)
])

## Reference
<p style="font-family: Arial; font-size:1.2em;color:black;">
Details of <a href = "https://www.tensorflow.org/api_docs/python/tf/keras/Sequential">Sequential Model</a>. Sequential provides training and predict features on this model.
</p>

### Functional API

<p style="font-family: Arial; font-size:1.2em;color:black;">You can also instantiate a Model with the <strong>"Functional API"</strong>, where you start from Input, you chain layer calls to specify the model's forward pass, and finally you create your model from inputs and outputs:

>import tensorflow as tf

>inputs = tf.keras.Input(shape=(2,))
    
>x = tf.keras.layers.Dense(4, activation=tf.nn.relu)(inputs)

>outputs = tf.keras.layers.Dense(2, activation=tf.nn.softmax)(x)

>model = tf.keras.Model(inputs=inputs, outputs=outputs)

    
<p style="font-family: Arial; font-size:1.2em;color:black;">You can also extend the model class, if love to live by sword!!
</p>

<p style="font-family: Arial; font-size:1.2em;color:black;">
    For each example the model returns a vector of "logits" or "log-odds" scores, one for each class.
</p>

<p style="font-family: Arial; font-size:1.2em;color:black;">
    <b>Unit Test</b>: Predict using untrained model.
</p>

In [ ]:
y_pred = model(X_train[:1])
y_pred

In [ ]:
tf.nn.softmax(y_pred)

The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 

In [ ]:
'''
Number of parameters:
    layer 1 : 2*4 +4 = 12 
    layer 2 : 4*4 + 2 = 10
We are not locking any layer hence all parameter are trainable    
'''

model.summary()

## Game Changer

`Softmax` function is prone to two issues: overflow and underflow.

- <b>Overflow</b> occurs when very large numbers are being fed.
- <b>Underflow</b> occurs when very small numbers (near zero) are being fed.

There are work arounds too, but, TF has its own approach.<br>
    
Intuition being that every system (32 / 64 bit precision) has limits in terms of largest or smallest value it can accurately process.

It also leads to the vanishing gradient in exponential functions. As you go higher/lower it  will overflow and the gradient will be `zero` irrespective of actually it being a `non-zero`.

## Lose Function

Our loss function is crossentropy as there are two or more label classes. The labels are provided using one-hot representation. We will use CategoricalCrossentropy loss. 
    
>y_true = [ 0, 2 ] <b>Sparse Representation</b><br>
>y_true = [ [1      0    0], [ 0    0   1 ] <b>Dense Representation</b><br>
>y_pred = [ [0.05, 0.95, 0], [0.1, 0.1, 0.8] ]


In case, you have not one-hot-encoded use `losses.SparseCategoricalCrossentropy`. It takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [ ]:
import sys

sys.float_info.max

In [ ]:
sys.float_info.min

In [ ]:
np.log(sys.float_info.max)

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy ( from_logits = True)

In [ ]:
y_train[:1]

In [ ]:
loss_fn(y_train[:1], y_pred).numpy()

<div class="alert alert-block alert-info" style="font-family: Arial; font-size:1.1em;">
    <b>Notes:</b>Change it to `adam` 
</div>

## Compile model
Using `model.compile`, specify the training configuration such as:
- Loss function to minimize
- Optimizer `'SGD'`
- List of metrics to monitor `'accuracy'`

In [ ]:
model.compile(optimizer='SGD',
    loss=loss_fn,
    metrics=['accuracy'])

<div class="alert alert-block alert-info" style="font-family: Arial; font-size:1.1em;">
    <b>Notes:</b>Add check point dir
</div>

## pass model

<div class="alert alert-block alert-info" style="font-family: Arial; font-size:1.1em;">
    <b>Notes:</b>add callback `callbacks=[model_callback]''
</div>


In [ ]:
history = model.fit(X_train, y_train, 
          validation_data=[X_test, y_test],
          epochs=EPOCHS
         )

## Evaluate the model
In case we want to separately evaluate this model, we have `evaluate` function.

In [ ]:
model.evaluate ( X_test,  y_test, verbose=2)

In [ ]:
hist_df = pd.DataFrame(history.history)
hist_df.head()

In [ ]:
fig, axes = plt.subplots(1,2 , figsize = (15,6))

ax = axes[0]

hist_df.plot(y = ['loss', 'val_loss'], ax = ax, colormap=CMAP)

ax.set_xlabel("Epochs")
ax.set_ylabel('Loss')
ax.set_title('Errors')
ax.grid();
ax = axes[1]

hist_df.plot( y = ['accuracy', 'val_accuracy'], ax = ax, colormap=CMAP)

ax.set_xlabel("Epochs")
ax.set_ylabel('Accuracy')
ax.grid()
ax.legend(loc = 'lower left')

plt.tight_layout()

<div class="alert alert-block alert-info" style="font-family: Arial; font-size:1.1em;">
    <b>Notes:</b>demonstrate function call
</div>

## Predictions
Build a prediction model using `tf.keras.Sequential` again and add `tf.keras.layers.Softmax` to model.

Make predictions and print classification reports and confusion matrix.

In [ ]:
prob_model = tf.keras.Sequential([
    model,
    tf.keras.layers.Softmax()
])

In [ ]:
y_pred = prob_model(X_train)

In [ ]:
print(classification_report(y_train, np.argmax(y_pred, axis = 1)))

In [ ]:
y_pred = prob_model(X_test).numpy()

print('Accuracy score on Test Data :{:.5f}'.format(accuracy_score(y_test, 
                                                                  np.argmax(y_pred, axis = 1))))

In [ ]:
print(classification_report(y_test, 
                            np.argmax(y_pred, axis = 1)))

<div class="alert alert-block alert-warning" style="font-family: Arial; font-size:1.1em;">
    <b>Notes:</b>Added Display labels
</div>

In [ ]:
cm  = confusion_matrix(y_test, np.argmax(y_pred, axis = 1))

disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=class_names.values())

fig, ax = plt.subplots(figsize = (4,4))

disp.plot(ax = ax, cmap = 'Blues', xticks_rotation = 'vertical', colorbar=False)

plt.show();

<div class="alert alert-block alert-info" style="font-family: Arial; font-size:1.1em;">
    <b>Notes:</b>add function to plot results data
</div>